In [ ]:
#
# Universidad EAFIT 
# 2025-2
# SI7016 - NLP - Lecture 07
#

In [ ]:
# instalar dependencias
%pip install transformers
%pip install openai

In [ ]:
# ejemplos de Lecture07 RAG

In [ ]:
# embeddings con HF
from sentence_transformers import SentenceTransformer

# Cargar modelo
model = SentenceTransformer("all-MiniLM-L6-v2")

# Crear embeddings
sentences = [
    "La capital de Francia es París.",
    "París es la ciudad más importante de Francia."
]
embeddings = model.encode(sentences)

print("Dimensiones:", embeddings.shape)


In [ ]:
# embeddings con OpenAI
from openai import OpenAI
client = OpenAI(api_key="OPENAI_API_KEY")

response = client.embeddings.create(
    model="text-embedding-3-small",
    input="La inteligencia artificial está transformando la educación."
)

vector = response.data[0].embedding
print("Dimensión:", len(vector))


In [ ]:
# FAISS (Facebook AI Similarity Search)
import faiss
import numpy as np

d = 768  # dimensión del embedding
index = faiss.IndexFlatL2(d)  # índice con distancia euclidiana

# Supongamos que tenemos embeddings en un array numpy
embeddings = np.random.random((100, d)).astype("float32")
index.add(embeddings)

query = np.random.random((1, d)).astype("float32")
distances, indices = index.search(query, k=5)
print(indices)


In [ ]:
# Chunking y ventanas de contexto
from langchain.text_splitter import RecursiveCharacterTextSplitter

text = "Un documento muy largo..."
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_text(text)

print("Número de chunks:", len(chunks))


In [ ]:
!pip install rank-bm25

In [ ]:
# sparce retrieval - bm25

from rank_bm25 import BM25Okapi

corpus = [
    "La capital de Francia es París",
    "París es conocida por la Torre Eiffel",
    "Roma es la capital de Italia"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

query = "¿Cuál es la capital de Francia?"
tokenized_query = query.split(" ")
scores = bm25.get_scores(tokenized_query)

print(scores)


In [ ]:
# dense retrieval

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Modelo de embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")

# Corpus
docs = [
    "París es la capital de Francia",
    "La Torre Eiffel está en París",
    "Roma es la capital de Italia"
]
doc_embeddings = model.encode(docs)

# Índice vectorial con FAISS
d = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(np.array(doc_embeddings))

# Consulta
query = "¿Dónde está la Torre Eiffel?"
q_emb = model.encode([query])
distances, indices = index.search(np.array(q_emb), k=2)

print("Documentos recuperados:", [docs[i] for i in indices[0]])


In [ ]:
# Rerankers
from sentence_transformers import CrossEncoder

model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

pairs = [
    ("¿Cuál es la capital de Francia?", "París es la capital de Francia"),
    ("¿Cuál es la capital de Francia?", "Roma es la capital de Italia")
]

scores = model.predict(pairs)
print(scores)


In [ ]:
# OTROS EJEMPLOS RAG

In [ ]:
# ejemplo - Uso de GPT-4 para Q&A Generativo
# actualización: https://github.com/openai/openai-python
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    #api_key="your-openai-api-key"
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "¿Quién descubrió la gravedad?",
        }
    ],
    model="gpt-4o",
)
print(response.choices[0].message.content)



In [ ]:
# Ejemplo: Q&A Mejorado con Recuperación de Información (RAG)
# se verá más adelante!!!!

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from openai import OpenAI

# Configurar OpenAI API Key
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    #api_key="your-openai-api-key"
)

# Cargar la base de datos de documentos en ChromaDB
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings())

# Crear un retriever para buscar información en la base de datos
retriever = vectorstore.as_retriever()

# Crear la Conversational Retrieval Chain con GPT-4
qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(model_name="gpt-4"),
    retriever=retriever
)

# Hacer una pregunta con recuperación de documentos
query = "¿Qué es LangChain?"
response = qa_chain({"question": query})
print(response["answer"])


In [ ]:
# summarization - resumen - Ejemplo con T5

from transformers import pipeline
summarizer = pipeline("summarization")

text = "Los modelos de lenguaje han cambiado la forma en que interactuamos con la tecnología..."
summary = summarizer(text, max_length=50, min_length=20, do_sample=False)

print(summary[0]['summary_text'])


In [ ]:
# Generación de texto - Ejemplo con GPT-4

import os
from openai import OpenAI

# Configurar OpenAI API Key
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    #api_key="your-openai-api-key"
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Escribe un poema sobre la inteligencia artificial.",
        }
    ],
    model="gpt-4o",
)
print(response.choices[0].message.content)



In [ ]:
# Chatbots y Asistentes Virtuales - Ejemplo con LangChain y GPT-4

from langchain.chat_models import ChatOpenAI

chatbot = ChatOpenAI(model_name="gpt-4")
response = chatbot.predict("¿Cuáles son los beneficios de la IA?")
print(response)


In [ ]:
# Ejemplo: Q&A Mejorado con Recuperación de Información (RAG)
# Uso de LangChain con ChromaDB
#
import openai
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

# Configurar OpenAI API Key
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    #api_key="your-openai-api-key"
)
# Cargar la base de datos de documentos en ChromaDB
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings())

# Crear un retriever para buscar información en la base de datos
retriever = vectorstore.as_retriever()

# Crear la Conversational Retrieval Chain con GPT-4
qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(model_name="gpt-4"),
    retriever=retriever
)

# Hacer una pregunta con recuperación de documentos
query = "¿Qué es LangChain?"
response = qa_chain({"question": query})
print(response["answer"])
